In [ ]:
# @hidden_cell
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

def set_hadoop_config_with_credentials_e452b25d17b0410187fa1d87a5c64a19(name):
    prefix = 'fs.swift.service.' + name
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + '.auth.url', 'https://identity.open.softlayer.com'+'/v3/auth/tokens')
    hconf.set(prefix + '.auth.endpoint.prefix', 'endpoints')
    hconf.set(prefix + '.tenant', 'dfff598f41234763839a71af72768b9e')
    hconf.set(prefix + '.username', '0e744535c0bf4886aa955dc73a972adb')
    hconf.set(prefix + '.password', 'gtJ]4QJ8SC?h5.e6')
    hconf.setInt(prefix + '.http.port', 8080)
    hconf.set(prefix + '.region', 'dallas')
    hconf.setBoolean(prefix + '.public', True)
name = 'keystone'
set_hadoop_config_with_credentials_e452b25d17b0410187fa1d87a5c64a19(name)

df_data_1 = sqlContext.read.format('com.databricks.spark.csv')\
    .options(header='true', inferschema='true')\
    .load("swift://sparkhw." + name + "/grades.csv")
    
df_data_2 = sqlContext.read.format('com.databricks.spark.csv')\
    .options(header='true', inferschema='true')\
    .load("swift://sparkhw." + name + "/names.csv")

    

In [116]:
#df_data_1.printSchema()
#df_data_2.columns
#df_data_2.dtypes
#df_data_2.explain
#df_data_1.rdd.sample(False,0.2).collect()
df_data_1.rdd.join(df_data_2.rdd).collect()
#df_data_1.rdd.saveAsTextFile("swift://sparkhw.keystone//a.txt")
#df_data_2.rdd.lookup(11)
#df_data_1.select(df_data_1["exam_1"]).collect()
#sc.parallelize([1, 2, 3]).sum()


[(8, (85, u'Dennis Lillee')),
 (12, (95, u'Rahul Dravid')),
 (4, (87, u'Garry Sobers')),
 (1, (76, u'Doug Walters')),
 (5, (69, u'Don Bradman')),
 (9, (23, u'Viv Richards')),
 (2, (90, u'Rohan Kanhai')),
 (10, (71, u'VVS Laxman')),
 (6, (90, u'Ian Chappell')),
 (11, (65, u'Michael Holding')),
 (3, (45, u'Peter Pollock')),
 (7, (81, u'Andy Roberts'))]

In [ ]:
#2a df_data_2.where(df_data_2.student_name.startswith("D") == True).count()
#2b 
#2c df_data_1.agg({"exam_1":"avg"}).collect()
#2d 


In [ ]:
#2c 
df_data_1.agg({"exam_1":"avg"}).collect()

In [ ]:
sqlContext.sql("select * from stu order by student_id asc").collect()

In [ ]:
df_data_1.registerTempTable("stu")
sqlContext.sql("select * from stu").collect()

In [118]:
ctable = df_data_1.join(df_data_2,df_data_1.student_id == df_data_2.student_id)
exam_average = ((ctable.exam_1+ctable.exam_2+ctable.exam_3)/3).alias("exam_average")
def mtog(mark):
    if mark > 89 : 
        return 'A' 
    elif mark > 79 :
        return 'B' 
    elif mark > 69 :
        return 'C'
    elif mark > 59 :
        return 'D'  
    else :
        return 'F'
gradefn = pyspark.sql.functions.udf(mtog)
grade = gradefn(exam_average).alias("grade")
dft = ctable.select(ctable.student_name,exam_average,grade)
#dft.orderBy(dft['exam_average'],ascending = False).show()
dft.sort(dft.exam_average.desc()).show()

+---------------+------------------+-----+
|   student_name|      exam_average|grade|
+---------------+------------------+-----+
|   Rahul Dravid| 94.66666666666667|    A|
|   Ian Chappell| 93.66666666666667|    A|
|   Rohan Kanhai| 92.66666666666667|    A|
|   Andy Roberts| 86.33333333333333|    B|
|  Dennis Lillee| 85.66666666666667|    B|
|   Garry Sobers| 85.66666666666667|    B|
|   Doug Walters|              79.0|    C|
|     VVS Laxman| 73.66666666666667|    C|
|    Don Bradman| 66.66666666666667|    D|
|Michael Holding| 65.66666666666667|    D|
|  Peter Pollock|41.333333333333336|    F|
|   Viv Richards|36.333333333333336|    F|
+---------------+------------------+-----+

